# Example SPARQL Queries to FNCT RDF Data

In this Jupyter Notebook, FNCT RDF data is loaded into a local ('emulated') triple store using the owlready2 package. Having the triple store ready by having included a knowlegde graph containing FNCT data, this graph / triple store can be queried. 

Therefor, 2 example SPARQL queries are given. They can be used to count the number of triples in the graph which may be useful for consistency checks and to query for some important information on FNCT experiments. In particular, the query provided addresses the process ID, material, medium, time to failure and actual tensile stress of all instances of type "FNCT" in the loaded dataset / graph (FNCT tests included) since this may be the most relevant results of FNCT experiments.

The working folder path has to be specified manually (see below, first cell): 
```python 
folder_path = r'Path_to_your_folder'
```

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import owlready2 as or2
from rdflib import Graph

# Path to the folder regarded has to be defined that contains FNCT data
# Caution: Exact folder to look for RDF data is set in the next lines - adapt if necessary
folder_path = r'Path_to_your_folder'

# Check if "graph_data" folder exists, otherwise stop reading in
graph_data_folder = os.path.join(folder_path, "graph_data")
if not os.path.exists(graph_data_folder):
    print(f"No graph data was found. '{graph_data_folder}' folder does not exist.")
    sys.exit()

# Create an empty graph
entire_knowledge_graph = Graph()

# Iterate over all RDF files in the given directory
for filename in os.listdir(graph_data_folder):
    if filename.endswith('.rdf'):
        file_path_rdf = os.path.join(graph_data_folder, filename)
        entire_knowledge_graph.parse(file_path_rdf, format='application/rdf+xml')

# Serialize the combined graph to an output file to facilitate reading using owlready2
output_file = os.path.join(graph_data_folder, "entire_knowledge_graph.rdf")
entire_knowledge_graph.serialize(destination=output_file, format='application/rdf+xml')
print(f"Combined graph serialized to {output_file}")


link_data = f"{graph_data_folder}\\entire_knowledge_graph.rdf"
link_core = "https://materialdigital.github.io/core-ontology/ontology.rdf" # Read in mid-level PMD Core Ontology (PMDco)
link_ontoFNCT = "https://markusschilling.github.io/ontoFNCT/ontology.rdf" # Read in OntoFNCT

triple_store = or2.World()
triple_store.get_ontology(link_core).load() # https://w3id.org/pmd/co
triple_store.get_ontology(link_ontoFNCT).load()  # https://w3id.org/ontofnct
triple_store.get_ontology(link_data).load()  # Local data mapped file

In [2]:
# Function to transform inputs to IRIs.
def to_iri(input):
    try:
        return input.iri
    except:
        pass
    return input

# Function to write the result of a SPARQL query into a (pandas) data frame.
def sparql_result_to_df(res):
    l = []
    for row in res:
        r = [ to_iri(item)  for item in row]
        l.append(r)
    return pd.DataFrame(l)

In [ ]:
# Query for the number of all triples in the loaded dataset / graph
query=("""
    PREFIX pmdco: <https://w3id.org/pmd/co/>
    PREFIX ontoFNCT: <https://w3id.org/ontofnct/>

    SELECT (COUNT(?s) AS ?count)
    WHERE {
       ?s ?p ?o
    }
    """
    )
# Create dataframe comprising the result of the SPARQL query.
res = triple_store.sparql(query)
data = sparql_result_to_df(res)
data

In [ ]:
# Query for the process ID, material, medium, time to failure and actual tensile stress of all instances of type "FNCT" in the loaded dataset / graph (FNCT tests included)
query=("""
    PREFIX pmdco: <https://w3id.org/pmd/co/>
    PREFIX ontoFNCT: <https://w3id.org/ontofnct/>

    SELECT DISTINCT ?processID ?material ?medium ?timeToFailure ?stressMeasured
    WHERE {
    ?p a ontoFNCT:FullNotchCreepTest .
    ?p pmdco:characteristic ?processIDInst .
    ?processIDInst a pmdco:ProcessIdentifier .
    ?processIDInst pmdco:value ?processID .
    ?s a pmdco:Specimen .
    ?p pmdco:input ?s .
    ?s pmdco:characteristic ?materialDesc .
    ?materialDesc a pmdco:materialDesignation .
    ?materialDesc pmdco:value ?material .
    ?p pmdco:participant ?mediumInst .
    ?mediumInst a pmdco:Medium .
    ?mediumInst pmdco:value ?medium .
    ?p pmdco:output ?tfInst .
    ?tfInst a ontoFNCT:TimeToFailure .
    ?tfInst pmdco:value ?timeToFailure .
    ?p pmdco:output ?stressInst .
    ?stressInst a ontoFNCT:MeasuredTensileStress .
    ?stressInst pmdco:value ?stressMeasured .
    } ORDER BY ?material ?medium ?stressMeasured
    """
    )
    # Create dataframe comprising the result of the SPARQL query. This may take some time depending on the complexity of the query.
res = triple_store.sparql(query)
data = sparql_result_to_df(res)
data